![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# **GenericClassifierApproach**

This notebook will cover the different parameters and usages of `GenericClassifierApproach`.

**📖 Learning Objectives:**

1. Understand how trains a TensorFlow model for generic classification of feature vectors.

2. Become comfortable using the different parameters of the annotator.


**🔗 Helpful Links:**

- Documentation : [GenericClassifierApproach](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier)

- Python Docs : [GenericClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/generic_classifier/generic_classifier/index.html)

- Scala Docs : [GenericClassifierApproach](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/generic_classifier/GenericClassifierApproach.html)

- For extended examples of usage, see the [Spark NLP Workshop repository](https://github.com/JohnSnowLabs/spark-nlp-workshop/tree/master/tutorials/Certification_Trainings/Healthcare).

## **🎬 Colab Setup**

In [ ]:
!pip install -q johnsnowlabs

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp

nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

spark = nlp.start()

👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.1.2, 💊Spark-Healthcare==5.1.2, running on ⚡ PySpark==3.1.2


## **🖨️ Input/Output Annotation Types**

- Input: `FEATURE_VECTOR`

- Output: `CATEGORY`

## **🔎 Parameters**


- `batchSize`: (int) Batch size

- `dropout`: (float) Dropout coefficient

- `epochsN`: (int) Maximum number of epochs to train

- `featureScaling`: (str) Feature scaling method. Possible values are 'zscore', 'minmax' or empty (no scaling)

- `fixImbalance`: (boolean) Fix the imbalance in the training set by replicating examples of under represented categories

- `labelColumn`: (str) Column with label per each document

- `learningRate`: (float) Learning Rate

- `modelFile`: (str) Location of file of the model used for classification

- `multiClass`: (boolean) If multiClass is set, the model will return all the labels with corresponding scores. By default, multiClass is false.

- `outputLogsPath`: (str) Folder path to save training logs. If no path is specified, the logs won't be stored in disk. The path can be a local file path, a distributed file path (HDFS, DBFS), or a cloud storage (S3).

- `validationSplit`: (float) The proportion of training dataset to be used as validation set.The model will be validated against this dataset on each Epoch and will not be used for training. The value should be between 0.0 and 1.0.

## Data Prepare

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/petfinder-mini.csv

In [ ]:
dataframe = pd.read_csv('petfinder-mini.csv')

In [ ]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

In [ ]:
dataframe = dataframe.drop(['AdoptionSpeed'], axis=1)

In [ ]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,target
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,1
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,1
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,1
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,1
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,1


In [ ]:
dataframe.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'target'],
      dtype='object')

In [ ]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Type          11537 non-null  object
 1   Age           11537 non-null  int64 
 2   Breed1        11537 non-null  object
 3   Gender        11537 non-null  object
 4   Color1        11537 non-null  object
 5   Color2        11537 non-null  object
 6   MaturitySize  11537 non-null  object
 7   FurLength     11537 non-null  object
 8   Vaccinated    11537 non-null  object
 9   Sterilized    11537 non-null  object
 10  Health        11537 non-null  object
 11  Fee           11537 non-null  int64 
 12  Description   11528 non-null  object
 13  PhotoAmt      11537 non-null  int64 
 14  target        11537 non-null  int64 
dtypes: int64(4), object(11)
memory usage: 1.3+ MB


In [ ]:
dataframe.target.value_counts()

1    8457
0    3080
Name: target, dtype: int64

In [ ]:
dataframe.Description = dataframe.Description.fillna('- no description -')

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

column_trans = make_column_transformer(
     (OneHotEncoder(), ['Type', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health']),
     (TfidfVectorizer(max_features=100,  norm='l2', ngram_range=(1, 3)), 'Description'),
     remainder=StandardScaler())

X = column_trans.fit_transform(dataframe.drop(['target'], axis=1))

y = dataframe.target

In [ ]:
y.nunique()

2

In [ ]:
X.shape

(11537, 302)

In [ ]:
input_dim = X.shape[1]

In [ ]:
input_dim

302

In [ ]:
df = pd.DataFrame.sparse.from_spmatrix(X)

df.columns = ['col_{}'.format(i) for i in range(input_dim)]

df['target']= y

df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_293,col_294,col_295,col_296,col_297,col_298,col_299,col_300,col_301,target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.452479,0.950288,-0.829762,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,-0.511871,1
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,-0.555981,-0.299388,1.077583,1
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.226480,0.0,-0.400729,1.575125,1.395473,1
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.198113,0.0,0.133816,0.181835,0.0,-0.555981,-0.299388,-0.193980,1


## Building a pipeline

The FeaturesAssembler is used to collect features from different columns. It can collect features from single value columns (anything which can be cast to a float, if casts fails then the value is set to 0), array columns or SparkNLP annotations (if the annotation is an embedding, it takes the embedding, otherwise tries to cast the 'result' field). The output of the transformer is a FEATURE_VECTOR annotation (the numeric vector is in the 'embeddings' field).

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [ ]:
spark_df = spark.createDataFrame(df)
spark_df.select(spark_df.columns[-10:]).show(2)

+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|col_293|col_294|col_295|col_296|col_297|col_298|            col_299|             col_300|            col_301|target|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0|-0.4524794726808656|  0.9502875792756131|-0.8297616989552165|     1|
|    0.0|    0.0|    0.0|    0.0|    0.0|    0.0| -0.555981017719065|-0.29938816657135553|-0.5118709929431844|     1|
+-------+-------+-------+-------+-------+-------+-------------------+--------------------+-------------------+------+
only showing top 2 rows



In [ ]:
import pyspark.sql.functions as F
spark_df.groupBy("target") \
    .count() \
    .orderBy(F.col("count").desc()) \
    .show()

+------+-----+
|target|count|
+------+-----+
|     1| 8457|
|     0| 3080|
+------+-----+



In [ ]:
(training_data, test_data) = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 9234
Test Dataset Count: 2303


In [ ]:
!pip install -q tensorflow tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 8.0 MB/s eta 0:00:00


In [ ]:
graph_folder = "gc_graph"

gc_graph_builder = medical.TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("target")\
    .setHiddenLayers([300, 200, 100])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph.pb")

In [ ]:
!mkdir logs

features_asm = medical.FeaturesAssembler()\
    .setInputCols(['col_{}'.format(i) for i in range(X.shape[1])])\
    .setOutputCol("features")

gen_clf = medical.GenericClassifierApproach()\
    .setLabelColumn("target")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(64)\
    .setFeatureScaling("zscore")\
    .setFixImbalance(True)\
    .setLearningRate(0.002)\
    .setOutputLogsPath("logs")\
    .setValidationSplit(0.2) # keep 20% of the data for validation purposes

clf_Pipeline = nlp.Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [ ]:
%%time

# train 50 epochs (takes around 1 min)

clf_model = clf_Pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph.pb
Build params: {'input_dim': 302, 'output_dim': 2, 'hidden_layers': [300, 200, 100], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}


Instructions for updating:
Colocations handled automatically by placer.


generic_classifier graph exported to gc_graph/gcf_graph.pb
CPU times: user 6.99 s, sys: 686 ms, total: 7.68 s
Wall time: 1min 21s


In [ ]:
import os
log_file_name = os.listdir("logs")[0]

with open("logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

Training 50 epochs
Epoch 1/50	1.05s	Loss: 30.216585	ACC: 0.62906086	Validation ACC: 0.6583649
Epoch 2/50	0.71s	Loss: 25.019773	ACC: 0.7180316	Validation ACC: 0.66269624
Epoch 3/50	0.88s	Loss: 22.155323	ACC: 0.7622276	Validation ACC: 0.6838116
Epoch 4/50	0.91s	Loss: 19.02265	ACC: 0.8038494	Validation ACC: 0.6876015
Epoch 5/50	0.87s	Loss: 15.515974	ACC: 0.84587526	Validation ACC: 0.71467245
Epoch 6/50	0.86s	Loss: 12.121668	ACC: 0.8855763	Validation ACC: 0.70925826
Epoch 7/50	0.83s	Loss: 9.027917	ACC: 0.92002416	Validation ACC: 0.7276665
Epoch 8/50	0.73s	Loss: 6.834477	ACC: 0.94426984	Validation ACC: 0.71467245
Epoch 9/50	0.42s	Loss: 5.1874356	ACC: 0.95945585	Validation ACC: 0.71467245
Epoch 10/50	0.51s	Loss: 3.241163	ACC: 0.97907186	Validation ACC: 0.7417434
Epoch 11/50	0.59s	Loss: 2.7199924	ACC: 0.98141164	Validation ACC: 0.7363292
Epoch 12/50	0.43s	Loss: 2.15634	ACC: 0.9865302	Validation ACC: 0.74120194
Epoch 13/50	0.49s	Loss: 2.0930836	ACC: 0.98599136	Validation ACC: 0.730915
Epoch 14

In [ ]:
pred_df = clf_model.transform(test_data)

In [ ]:
pred_df.select('target','prediction.result').show()

+------+------+
|target|result|
+------+------+
|     1|   [1]|
|     1|   [0]|
|     0|   [1]|
|     1|   [0]|
|     0|   [0]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     0|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
|     1|   [1]|
+------+------+
only showing top 20 rows



In [ ]:
preds_df = pred_df.select('target','prediction.result').toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : int(x[0]))


In [ ]:
preds_df.sample(10)

,target,result
1504,0,0
1213,1,0
1063,0,1
837,0,1
1021,1,0
1540,0,1
1221,1,1
1492,0,1
2141,0,1
1579,0,0


In [ ]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report, accuracy_score

print (classification_report(preds_df['target'], preds_df['result'], digits=4))

print (accuracy_score(preds_df['target'], preds_df['result']))


              precision    recall  f1-score   support

           0     0.5217    0.4953    0.5081       632
           1     0.8127    0.8282    0.8204      1671

    accuracy                         0.7369      2303
   macro avg     0.6672    0.6617    0.6643      2303
weighted avg     0.7328    0.7369    0.7347      2303

0.7368649587494572
